# Regression and Gauss-Markov assumptions

For this exercise, we recommend using `statsmodels`. 
Unless you haven't already installed it, you can do so by running
```bash
pip install statsmodels
```

*Make sure you have activated your `baml-venv` environment before doing so!*

## Imports and data loading

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv("gauss-markov.csv")
data.head(10)

## a)

We start by using the simple linear regression model
$$
Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \beta_3 X_3
$$

Using [``sm.OLS``](https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLS.html), compute optimal values for the parameters.

> Note: You may want to use ``sm.add_constant`` to add values for the intercept.



In [ ]:
# Prepare input data
X = sm.add_constant(data[["x1", "x2", "x3"]])
y = data["y"]

In [ ]:
# Fit a linear model with statsmodels
model = sm.OLS(y, X)
results = model.fit()

In [ ]:
# Show the results using the summary() function
print(results.summary())

Use the model to predict the $y$-values

In [ ]:
predicted_values = model.predict(results.params, X)

In [ ]:
# Visualization of the predicted variables vs. the true variables
fig, axs = plt.subplots(1, 3, figsize=(20, 5))
for ax, variable_name in zip(axs, ["x1", "x2", "x3"]):
    ax.scatter(data[variable_name], data["y"], label="Ground truth")
    ax.scatter(data[variable_name], predicted_values, label="Model prediction")
    ax.legend()
    ax.set_xlabel(variable_name)

## b)

Compute the residuals $e = \hat{y} - y$ of the resulting model.

In [ ]:
residuals = data["y"] - predicted_values
# alternatively: residuals = results.resid

Plot the residuals over the input variables $x_1$ and $x_2$. What do you observe?

In [ ]:
plt.figure()
plt.scatter(data["x1"], residuals)
plt.figure()
plt.scatter(data["x2"], residuals)
plt.figure()
plt.scatter(data["x3"], residuals)

Using a White test ([`statsmodels.stats.diagnostic.het_white`](https://www.statsmodels.org/dev/generated/statsmodels.stats.diagnostic.het_white.html)), show that we can reject the hypothesis of homoscedastic residuals at an $\alpha$ level of 0.01.

In [ ]:
from statsmodels.stats.diagnostic import het_white

statistic, p_value, _, _ = het_white(residuals, X)
print(f"Value of the null-hypothesis that the residuals are homoscedastic: {statistic}")
print(f"p-value of the statistic: {p_value}")

## c)

Consider the alternative model
$$
Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \beta_3 X_3 + \beta_4 X_1^2
$$

Compute the optimal parameter values. You should observe that the $R^2$ value improves drastically over the previous model.

In [ ]:
# Prepare input data
X = sm.add_constant(data[["x1", "x2", "x3"]])
X["x1^2"] = np.square(X["x1"])
y = data["y"]

# Fit a linear model
model = sm.OLS(y, X)
results = model.fit()

print(results.summary())

Although this model gives a very good fit of the data, there is another problem.
Use the Variance inflation factor ([`statsmodels.stats.outliers_influence.variance_inflation_factor`](https://www.statsmodels.org/dev/generated/statsmodels.stats.outliers_influence.variance_inflation_factor.html)) to check whether the variables are dependent.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

for index, variable_name in enumerate(X.columns):
    if variable_name == "const": 
        continue
    print(f"VIF for variable {variable_name} is {vif(X, index)}")

In [ ]:
# Bonus: Check if residuals are now homoscedastic
from statsmodels.stats.diagnostic import het_white

statistic, p_value, _, _ = het_white(results.resid, X)
print(f"Value of the null-hypothesis that the residuals are homoscedastic: {statistic}")
print(f"p-value of the statistic: {p_value}")

## d)
Consider a third model:
$$
Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \beta_3 X_1^2
$$

Compute the optimal parameter values.

In [ ]:
# Prepare input data
X = sm.add_constant(data[["x1", "x2"]])
X["x1^2"] = np.square(X["x1"])
y = data["y"]

# Fit a linear model
model = sm.OLS(y, X)
results = model.fit()

print(results.summary())

Check if the model has multicollinear input variables using the VIF.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

for index, variable_name in enumerate(X.columns):
    if variable_name == "const": 
        continue
    print(f"VIF for variable {variable_name} is {vif(X, index)}")

Check if the model satisfies the homoscedasticity assumption using the White test and an $\alpha$ level of 0.01.

In [ ]:
from statsmodels.stats.diagnostic import het_white

statistic, p_value, _, _ = het_white(results.resid, X)
print(f"Value of the null-hypothesis that the residuals are homoscedastic: {statistic}")
print(f"p-value of the statistic: {p_value}")

In [ ]:
# Bonus: Visualization of the residuals
plt.figure()
plt.scatter(data["x1"], results.resid)
plt.figure()
plt.scatter(data["x2"], results.resid)

In [ ]:
# Visualization of the predicted variables vs. the true variables
fig, axs = plt.subplots(1, 3, figsize=(20, 5))
for ax, variable_name in zip(axs, ["x1", "x2", "x3"]):
    ax.scatter(data[variable_name], data["y"], label="Ground truth")
    ax.scatter(data[variable_name], model.predict(results.params, X), label="Model prediction")
    ax.legend()
    ax.set_xlabel(variable_name)